## Bugs 홈페이지에서 실시간 순위차트 크롤링
- 순위, 제목, 가수, 앨범명을 추출하여 DataFrame 형태로 만들기
- 원하는 날짜로 추출가능하게

In [77]:
from bs4 import BeautifulSoup
from calendar import monthrange
import pandas
import requests
import time

def extract_artist(tr): # 아티스트 추출 함수
    artist_cnt = len(tr.find('p',{'class':'artist'}).find_all('a'))
    if artist_cnt == 1:
        artist = tr.find('p',{'class':'artist'}).text.strip()
    else: # 아티스트 여러명일 때
        artist = tr.find('p',{'class':'artist'}).find('a',{'class':'more'}).get('onclick')
        alist = artist[artist.index("'"):].split('||')
        artist = []
        for index in range(1,len(alist), 3):
            artist.append(alist[index])
        artist = "/".join(artist)
    return artist

def extract_rank(tr): # 순위 추출 함수
    rank = tr.find('strong').text
    return rank

def extract_title(tr): # 타이틀 추출 함수 (예외처리, 음원철수)
    try:
        title = tr.find('p',{'class':'title'}).find('a').text.strip()
    except:
        title = tr.find('p',{'class':'title'}).find('span','').text.strip()
    return title

def extract_album(tr): # 앨범 추출 함수 (예외처리, 음원철수)
    try:
        album = tr.find('a',{'class':'album'}).text.strip()
    except:
        album = tr.find('span',{'class':'album'}).text.strip()
    return album

def make_html(date):    
    url = f"https://music.bugs.co.kr/chart/track/realtime/total?chartdate={date}"
    response = requests.get(url)
    content = response.content
    html = BeautifulSoup(content, 'html.parser')
    return html

In [89]:
for year in range(2020, 2021, 1):  
    for month in range(12, 13, 1):
        days= monthrange(year,month)[1] + 1 # 월마다 일수가 다르기 때문에
        writer = pandas.ExcelWriter(f'bugs_{year}.{month:02}.xls')   # 엑셀 한 파일에 시트별로 저장해주기
        for day in range(1, 6, 1):
            date = f'{year}{month:02}{day:02}' # f문자열로 날짜를 항상 8자리로 만들어줌
            html = make_html(date)
            table = html.find('table', {'class':"list trackList byChart"})
            tr_list = table.find('tbody').find_all('tr')
            rows = []
            for tr in tr_list:
                rank = extract_rank(tr)
                title = extract_title(tr)
                artist = extract_artist(tr)
                album = extract_album(tr)
                rows.append([rank,title,artist,album]) 
                
            df = pandas.DataFrame(rows) # 추출한 것을 판다스를 활용해, 데이터프레임으로 만들기
            df.columns=['rank','title','artist','album']
            df.set_index('rank',inplace=True)
            df.to_excel(writer, sheet_name = date)
            time.sleep(3)
    writer.save()           

In [ ]:
"""
filename = f'bugs_{date}.xls'
df.to_excel(filename)
"""
